In [1]:
import copy
import os
import json
import traceback
import requests
from collections import defaultdict
import importlib
import contextlib

import numpy as np
from datetime import date
import py3Dmol
import networkx as nx
from pdbeccdutils.core import ccd_reader
import rdkit
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_entry_cif(pdb_id):
    """
    Downloads and writes PDB entry updated mmCIF to a file and 
    returns the path

    Args:
        pdb_id (str): PDB id of an entry

    Returns:
        str: Path to the downloaded file
    """
    ENTRY_URL = "https://files.rcsb.org/ligands/download/"
    entry_url = os.path.join(ENTRY_URL, f"{pdb_id}.cif")
    response = requests.get(entry_url, stream=True)
    response.raise_for_status()
    with open(f"{pdb_id}.cif", "wb") as fh:
        fh.write(response.content)

    return f"{pdb_id}.cif"

def view3D(molecule):
    """Format 3D view of RDKit molecule

    Args:
        molecule (rdkit.Chem.rdchem.Mol): RDKit molecule to be viewed

    Returns:
        py3Dmol.view: View object.
    """
    mol = Chem.Mol(molecule)
    view = py3Dmol.view(
        data=Chem.MolToMolBlock(mol),  # Convert the RDKit molecule for py3Dmol
        style={"stick": {}, "sphere": {"scale": 0.3}}
    )
    return view

In [2]:
conformers = []
with open(f"conformers.txt", 'r') as f:
    for line in f:
        conformers.append(line.strip().split(", "))

print(f"There are {len(conformers)} molecules expected to have multiple conformers")
print([len(x) for x in conformers])

There are 4 molecules expected to have multiple conformers
[2, 2, 2, 2]


In [3]:
conformers_rdmol = []
for labels in conformers:
    conformers_rdmol.append([])
    for label in labels:
        code = label.split("_")[1]
        rdmol = ccd_reader.read_pdb_cif_file(f"cifs/{code}.cif", sanitize=True).component.mol
        if rdmol.GetNumConformers() > 1:
            rdmol.RemoveConformer(0) # Remove openeye coord estimate
        
        conformers_rdmol[-1].append(rdmol)
        
conformers_rdmol.sort(key=len)

## Molecule 1

These are isomers, where the amine terminated chain has a N-O-Metal across from a C-O-Metal bond, while the second structures has a N-O-Metal bond.

In [4]:
mols = conformers_rdmol[0]
view3D(mols[0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
view3D(mols[1])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Molecule 2

These are isomers, not conformers

In [6]:
mols = conformers_rdmol[1]
view3D(mols[0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
view3D(mols[1])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.